### Importar Librerías 

In [1]:
%reset -f
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
from functools import partial, reduce
import operator
import re
import time
from bs4 import BeautifulSoup
from collections import Counter
from wordcloud import WordCloud


In [4]:
#Functions

In [6]:
def getAllPaths(data_directory):
    #Input: directorio de los archivos.
    #Return: Arreglo con las rutas de los archivos.
    file_name = [os.path.join(data_directory,f) 
                 for f in os.listdir(data_directory)
                 if f.endswith(".csv")]
    return file_name

def sentences_ravel(df):
    sentences = df.to_list()
    return " ".join(sentences)

def cargarDF(x,df,col="PATH_FILES"):
    data =(df.iloc[x,0:1].values[0])
    return pd.read_csv(data)

def reducirDF(x,df,colname):
    return df[x].iloc[:,5:6]

def dropNA(x,df):
    data = df.copy()
    data[x].replace('', np.nan)
    data[x]["text"].str.lower().str.strip();
    return data[x].dropna()

### Cargo todos los dataset de pruebas y los concatenamos.

In [ ]:
#df = df.sample(frac=0.75)

In [ ]:
#df.reset_index(drop=True, inplace=True)

In [ ]:
#plt.figure(figsize=(20,20))
#plt.imshow(WordCloud().generate(sentences_as_one_string))

### 


In [7]:
PATHS_CSV = getAllPaths("../data_sophia2/")
PATH_DF = pd.DataFrame(PATHS_CSV)
PATH_DF.columns=["PATH_FILES"]
PATH_DF.head()

,PATH_FILES
0,../data_sophia2/chile_2020-11.csv
1,../data_sophia2/chile_2021-02.csv
2,../data_sophia2/chile_2021-03.csv
3,../data_sophia2/chile_2020-10.csv
4,../data_sophia2/chile_2021-04.csv


### cargar los datasets

In [8]:
df__sophia2 = list(map(partial(cargarDF, df=PATH_DF), range(PATH_DF.shape[0])))
df__sophia2[0].head()

,id,country,media_outlet,url,title,text,date,year,id_journalist
0,23046,chile,horas24,https://www.24horas.cl/nacional/adolescente-ba...,Adolescente baleado por carabinero tras intent...,Fue la noche del pasado viernes cuando tres su...,2020-11-01,2020.0,NaN
1,23062,chile,horas24,https://www.24horas.cl/nacional/senal-en-vivo-...,Señal en vivo: Cristián Warnken conversa con E...,"Este domingo tendremos una nueva edición de ""E...",2020-11-01,2020.0,NaN
2,23082,chile,horas24,https://www.24horas.cl/nacional/ministro-bello...,Ministro Bellolio: “El Gobierno está ocupado y...,"La tarde este domingo, el ministro Vocero de G...",2020-11-01,2020.0,NaN
3,23101,chile,horas24,https://www.24horas.cl/nacional/fueron-rescata...,Fueron rescatados de emergencia: Ciclistas se ...,NaN,2020-11-01,2020.0,NaN
4,23112,chile,horas24,https://www.24horas.cl/nacional/bellolio-para-...,"Bellolio: ""Para que la Convención Constituyent...",NaN,2020-11-01,2020.0,NaN


### Seleccionar los columnas de texto

In [9]:
df_sample_sophia2 =list(map(partial(reducirDF, df=df__sophia2,colname="text"),range(len(df__sophia2))))

In [10]:
#eliminar NaN
df_sophia2_subset_pd = list(map(partial(dropNA,df=df_sample_sophia2),
                                range(len(df_sample_sophia2))))

In [15]:
df_sophia2_subset_pd[0]

,text
0,Fue la noche del pasado viernes cuando tres su...
1,"Este domingo tendremos una nueva edición de ""E..."
2,"La tarde este domingo, el ministro Vocero de G..."
5,El Ministerio de Salud entregó este domingo un...
6,Durante la mañana de este domingo el President...
...,...
13528,Luego de la muerte de Diego Armando Maradona ...
13529,Este domingo el Barcelona goleó por 4-0 al Osa...
13530,Tomás Barrios tuvo una excelente semana y alc...
13531,Las dobles faltas son errores comunes en el te...


### juntar los Dataframes individuales

In [17]:
df_concat = pd.concat(df_sophia2_subset_pd, ignore_index=True)
df_concat.to_csv("./data_noprocesada.csv")

### StopWords y signos de puntuación

In [18]:
import nltk
from nltk.corpus import stopwords
import string
nltk.download('stopwords')
punct = string.punctuation
stop_words_en = stopwords.words('spanish')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/icarlos/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [20]:
import ipyparallel as ipp
rc = ipp.Client()
dv=rc[:]

In [21]:
def remove_stopword(x,stop_words_list,punc):
    palabra = [char for char in x if char not in punc]
    palabra = ''.join(palabra)
    if(palabra.lower() not in stop_words_list):
            return palabra


In [23]:
#1 core
import time
start = time.time()
text = list(filter(None,list(map(partial(remove_stopword,
                                        stop_words_list=stop_words_en,
                                       punc=punct),df_concat["text"]))))
print(time.time()-start)

5.684140920639038


In [22]:
#10 cores
import time
start = time.time()
text = list(filter(None,list(dv.map(partial(remove_stopword,
                                        stop_words_list=stop_words_en,
                                       punc=punct),df_concat["text"]))))
print(time.time()-start)

1.9430596828460693


### Guardar  datos limpios

In [ ]:
#data_clean = pd.DataFrame(text, columns =['text'])
#data_clean.to_csv("./data_clean.csv")

In [24]:
data_clean = pd.read_csv("data_clean.csv")

In [26]:
df = data_clean.iloc[:1000,1:2]
df.head()

,text
0,Fue la noche del pasado viernes cuando tres su...
1,Este domingo tendremos una nueva edición de En...
2,La tarde este domingo el ministro Vocero de Go...
3,El Ministerio de Salud entregó este domingo un...
4,Durante la mañana de este domingo el President...
